In [45]:
# packs
import os
import fnmatch
import numpy as np
import pandas as pd
from google.cloud import bigquery
import warnings

# ocultar warnings
warnings.filterwarnings('ignore')

#funciones
def consulta_archivo(patron, ruta='../querys'):
    """
    Ejecuta una consulta en BigQuery utilizando un archivo SQL que coincide con un patrón.

    Parameters:
    - patron (str): El patrón para buscar el archivo SQL.
    - ruta (str, optional): La ruta del directorio donde se buscarán los archivos SQL. 
                           Por defecto, es '../querys'.

    Returns:
    - pandas.DataFrame or None: El resultado de la consulta en forma de DataFrame de pandas. 
                               Si no se encuentra el archivo SQL, retorna None.

    Example:
    ```python
    patron_busqueda = 'megabrands'
    df_resultado = consulta_archivo(patron_busqueda)
    ```

    """
    # Encuentra el archivo SQL utilizando el patrón
    for nombre_archivo in os.listdir(ruta):
        if fnmatch.fnmatch(nombre_archivo, f'*{patron}*.sql'):
            # Construye la ruta completa al archivo SQL
            ruta_completa_sql = os.path.join(ruta, nombre_archivo)
            print(nombre_archivo)
            # Lee el contenido del archivo SQL
            with open(ruta_completa_sql, "r") as file:
                sql = file.read()

            # Ejecuta la consulta y convierte los resultados a un DataFrame de pandas
            cliente = bigquery.Client()
            df_resultado = cliente.query(sql).to_dataframe()
            print(df_resultado.head(2))
            print(df_resultado.shape)
            return df_resultado

    # Si no se encuentra ningún archivo que coincida con el patrón
    print('No se encontró el archivo SQL.')
    return None

def procesar_df_interacciones(df):
    """
    Función que procesa un DataFrame según las especificaciones dadas.

    Parámetros:
    - df: DataFrame a procesar.

    Returns:
    - DataFrame procesado.
    """
    print('Volumen inicial:')
    print(df.shape)
    print('Fase 1: Eliminando la variable "abi_brand"')
    df = df.drop('abi_brand', axis=1, errors='ignore')
    print('Fase 1 Done\n')

    print('Fase 2: Filtrando valores superiores al percentil 95 en "nInteracciones" y "Dias_interaccion"')
    limite_superior_nInteracciones = df['nInteracciones'].quantile(0.95)
    limite_superior_Dias_interaccion = df['Dias_interaccion'].quantile(0.95)
    
    df = df[(df['nInteracciones'] <= limite_superior_nInteracciones) & (df['Dias_interaccion'] <= limite_superior_Dias_interaccion)]
    
    print('Fase 2 Done\n')

    print('Fase 3: Seleccionando registros con "nInteracciones" > 1')
    df = df[df['nInteracciones'] > 1]
    print('Fase 3 Done\n')

    print('Fase 4: Aplicando la condición "brand lovers"')
    df = df[(df['nInteracciones'] > 4) | (df['Dias_interaccion'] < 75)]
    print('Fase 4 Done\n')

    print('Volumen final:')
    print(df.shape)
    df1 = df[['td_id1']] 
    df1.loc[:, 'origen'] = 'interaccion'
    return df1


In [16]:
# Llama a la función con el nombre de tu archivo SQL
df = consulta_archivo('nes_mega')
# Agrupar corona
df.loc[df['abi_brand'] == 'Corona Extra', 'abi_brand'] = 'Corona'

interacciones_mega.sql
                                              td_id1  nInteracciones  \
0  ffffac8e608e2c257507de669d5e84d62cccab4ce8653e...               2   
1  ffffa7a98bfb8d9dcfe437b51cc4b0f0bf36f806f55faf...               1   

   Dias_interaccion     abi_brand  
0               131        Corona  
1               104  Corona Extra  
(808618, 4)


In [19]:
df.abi_brand.value_counts()

abi_brand
Corona           248196
Aguila           244624
Poker            199310
Club Colombia    116488
Name: count, dtype: int64

In [23]:

# Lista de marcas
marcas = ['Club Colombia', 'Poker', 'Corona', 'Aguila']

# Diccionario para almacenar DataFrames filtrados
dataframes_filtrados = {marca: df[df['abi_brand'].str.contains(marca)] for marca in marcas}

# Accede a los DataFrames individuales usando el nombre de la marca
club_df = dataframes_filtrados['Club Colombia']
poker_df = dataframes_filtrados['Poker']
corona_df = dataframes_filtrados['Corona']
aguila_df = dataframes_filtrados['Aguila']

In [46]:
club_lovers = procesar_df_interacciones(club_df)

Volumen inicial:
(116488, 4)
Fase 1: Eliminando la variable "abi_brand"
Fase 1 Done

Fase 2: Filtrando valores superiores al percentil 95 en "nInteracciones" y "Dias_interaccion"
Fase 2 Done

Fase 3: Seleccionando registros con "nInteracciones" > 1
Fase 3 Done

Fase 4: Aplicando la condición "brand lovers"
Fase 4 Done

Volumen final:
(19953, 3)


In [47]:
club_lovers.head()

,td_id1,origen
23,fffd89f1-da0a-414a-91c3-8f7d395c2c31,interaccion
47,fffb6838-c5d4-40e4-b9de-8af0635dff9f,interaccion
75,fff8cf9c7c603e23e5aadd03a5ed78aec24707f87ad873...,interaccion
88,fff7bfd701a95926596a72ae8e160a18cafb5015208463...,interaccion
197,ffeef0605759c9565dba5df3c5eade575b16ebd83ed0fd...,interaccion


In [48]:
club_lovers = procesar_df_interacciones(club_df)
poker_lovers = procesar_df_interacciones(poker_df)
corona_lovers = procesar_df_interacciones(corona_df)
aguila_lovers = procesar_df_interacciones(aguila_df)

Volumen inicial:
(116488, 4)
Fase 1: Eliminando la variable "abi_brand"
Fase 1 Done

Fase 2: Filtrando valores superiores al percentil 95 en "nInteracciones" y "Dias_interaccion"
Fase 2 Done

Fase 3: Seleccionando registros con "nInteracciones" > 1
Fase 3 Done

Fase 4: Aplicando la condición "brand lovers"
Fase 4 Done

Volumen final:
(19953, 3)
Volumen inicial:
(199310, 4)
Fase 1: Eliminando la variable "abi_brand"
Fase 1 Done

Fase 2: Filtrando valores superiores al percentil 95 en "nInteracciones" y "Dias_interaccion"
Fase 2 Done

Fase 3: Seleccionando registros con "nInteracciones" > 1
Fase 3 Done

Fase 4: Aplicando la condición "brand lovers"
Fase 4 Done

Volumen final:
(59667, 3)
Volumen inicial:
(248196, 4)
Fase 1: Eliminando la variable "abi_brand"
Fase 1 Done

Fase 2: Filtrando valores superiores al percentil 95 en "nInteracciones" y "Dias_interaccion"
Fase 2 Done

Fase 3: Seleccionando registros con "nInteracciones" > 1
Fase 3 Done

Fase 4: Aplicando la condición "brand lovers

In [ ]:
# cortes para tada